# Syntax preprocessing

This tutorial describes the tools that are needed to convert the morphologically analysed Estonian text into the format in which it can be parsed with **VISL-CG3 parser**. VISL CG3 is a rule-based syntactic parser that has thousands of Estonian-specific handcrafted rules for tagging syntactic functions and dependencies. However, the parser needs a little more information than is given out by the standard morphological analyser (e.g. pronoun types, verb subcategorization, etc.), and its input needs to be in a different format. Therefore, several taggers, rewriters, and an exporter that are described in the rest of the tutorial are needed.

## Syntax preprocessing taggers and rewriters


|tagger|rewriters|source attributes|target attributes|values|
|------|---------|-----------------|-----------------|------|
|&nbsp;| PunctuatinonTypeRewriter | partofspeech, root|punctuation_type|```None```, 'Fst', 'Com', 'Col', ... |
|&nbsp;| MorphToSyntaxMorphRewriter|partofspeech, form|partofspeech, form||
|PronounTypeTagger|PronounTypeRewriter|root, ending, clitic, partofspeech|pronoun_type| ```None```, ('det',), ('pers ps3',), ('pos', 'det', 'refl'), ... |
|&nbsp;|RemoveDuplicateAnalysesRewriter|root, ending, clitic, partofspeech, form|&nbsp;|&nbsp;|
|&nbsp;|RemoveAdpositionAnalysesRewriter|partofspeech, form|&nbsp;|&nbsp;|
|&nbsp;|LetterCaseRewriter|word_text|cap|```None```, 'cap'|
|FiniteFormTagger|MorphToSyntaxMorphRewriter, FiniteFormRewriter|partofspeech, form|fin|```None```, ```True```, ```False```|
|VerbExtensionSuffixTagger|VerbExtensionSuffixRewriter|root|verb_extension_suffix|```None```,'tud','nud','mine','nu','tu','v','tav','mata','ja'|
|SubcatTagger|MorphToSyntaxMorphRewriter, SubcatRewriter|root, partofspeech, form|subcat|```None```, 'Intr', 'Part', 'gen', ...|
|MorphExtendedTagger|PunctuatinonTypeRewriter, MorphToSyntaxMorphRewriter, PronounTypeRewriter, RemoveDuplicateAnalysesRewriter, RemoveAdpositionAnalysesRewriter, LetterCaseRewriter, FiniteFormRewriter, VerbExtensionSuffixRewriter, SubcatRewriter|root, ending, clitic, partofspeech, form|partofspeech, form, punctuation_type, pronoun_type, cap, fin, verb_extension_suffix, subcat ||

In [1]:
from estnltk import Text
from estnltk.common import abs_path

## PronounTypeRetagger

Create an instance of `PronounTypeRetagger`.

In [2]:
# NBVAL_IGNORE_OUTPUT
from estnltk.taggers import PronounTypeRetagger

retagger = PronounTypeRetagger('morph_analysis')
retagger

PronounTypeRetagger(check_output_consistency=False, pronoun_types=defaultdict(<class 'tuple'>, {'emb-kumb': ('det',), 'eikeegi': ('indef',), 'eimiski': ('indef',), 'esimene': ('dem',), 'iga': ('det',), 'igasugune': ('dem',), 'igaüks': ('det',), 'ise': ('pos', 'det', 'refl'), 'iseenese': ('refl',), 'isesugune': ('dem',), 'keegi': ('indef',), 'kes': ('inter_rel',), 'kõik': ('det',), 'kumb': ('rel',), 'kumbki': ('det',), 'mäherdune': ('indef',), 'meiesugune': ('dem',), 'meietaoline': ('dem',), 'mihuke': ('inter_rel',), 'mihukene': ('inter_rel',), 'milletaoline': ('dem',), 'milline': ('rel',), 'mina': ('ps1',), 'minakene': ('dem',), 'mingi': ('indef',), 'mingisugune': ('indef',), 'minusugune': ('dem',), 'minutaoline': ('dem',), 'mis': ('inter_rel',), 'miski': ('inter_rel',), 'miskisugune': ('inter_rel',), 'missugune': ('inter_rel',), 'misuke': ('dem',), 'misukene': ('dem',), 'mitmendik': ('inter_rel',), 'mitmes': ('inter_rel', 'indef'), 'mitmesugune': ('indef',), 'mitmetaoline': ('indef',), 'mitu': ('indef',), 'mõlema': ('det',), 'mõnesugune': ('indef',), 'mõni': ('indef',), 'mõningane': ('indef',), 'mõningas': ('indef',), 'muist': ('indef',), 'muu': ('indef',), 'naasugune': ('dem',), 'nendesugune': ('dem',), 'nendetaoline': ('dem',), 'nihuke': ('dem',), 'nihukene': ('dem',), 'niimitu': ('indef', 'inter_rel'), 'niipaljuke': ('indef',), 'niisamasugune': ('dem',), 'niisugune': ('dem',), 'nisuke': ('dem',), 'nisukene': ('dem',), 'oma': ('pos', 'det', 'refl'), 'omaenese': ('pos',), 'omasugune': ('dem',), 'omataoline': ('dem',), 'palju': ('indef',), 'paljuke': ('indef',), 'säärane': ('dem',), 'säherdune': ('dem',), 'sama': ('dem',), 'samasugune': ('dem',), 'samataoline': ('dem',), 'samune': ('dem',), 'see': ('dem',), 'seesama': ('dem',), 'seesamane': ('dem',), 'seesamune': ('dem',), 'seesinane': ('dem',), 'seesugune': ('dem',), 'selletaoline': ('dem',), 'selline': ('dem',), 'setu': ('indef',), 'sihuke': ('dem',), 'sihukene': ('dem',), 'sina': ('ps2',), 'sinusugune': ('dem',), 'sinutaoline': ('dem',), 'taoline': ('dem',), 'teiesugune': ('dem',), 'teietaoline': ('dem',), 'teine': ('dem',), 'teineteise': ('rec',), 'teistsugune': ('dem',), 'tema': ('ps3',), 'temake': ('ps3',), 'temakene': ('ps3',), 'temasugune': ('dem',), 'temataoline': ('dem',), 'too': ('dem',), 'toosama': ('dem',), 'üks': ('dem', 'indef'), 'üksteise': ('rec', 'indef')}))

Run retagger and observe that the `pronoun_type` attribute is added to the `morph_analysis` layer.

In [3]:
text = Text('Kumb, sina või mina?').tag_layer()

retagger.retag(text)
text.morph_analysis

Layer(name='morph_analysis', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech', 'pronoun_type'), spans=SL[Span('Kumb', [{'normalized_text': 'Kumb', 'lemma': 'kumb', 'root': 'kumb', 'root_tokens': ['kumb'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'P', 'pronoun_type': ('rel',)}]),
Span(',', [{'normalized_text': ',', 'lemma': ',', 'root': ',', 'root_tokens': [','], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z', 'pronoun_type': None}]),
Span('sina', [{'normalized_text': 'sina', 'lemma': 'sina', 'root': 'sina', 'root_tokens': ['sina'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'P', 'pronoun_type': ('ps2',)}]),
Span('või', [{'normalized_text': 'või', 'lemma': 'või', 'root': 'või', 'root_tokens': ['või'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'J', 'pronoun_type': None}]),
Span('mina', [{'normalized_text': 'mina', 'lemma': 'mina', 'root': 'mina', 'root_tokens': ['mina'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S', 'pronoun_type': None}]),
Span('?', [{'normalized_text': '?', 'lemma': '?', 'root': '?', 'root_tokens': ['?'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z', 'pronoun_type': None}])])

## FiniteFormTagger

In [4]:
from estnltk.taggers import FiniteFormTagger

fsToSyntFulesFile = abs_path('taggers/standard/syntax/preprocessing/rules_files/tmorftrtabel.txt')

tagger = FiniteFormTagger('morph_analysis', fs_to_synt_rules_file=fsToSyntFulesFile)
tagger

FiniteFormTagger(input_layers=('morph_analysis',), output_layer=morph_extended, output_attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form',  ..., type: <class 'tuple'>, length: 9, morph_to_syntax_morph_tagger=MorphToSyntaxMorphRetagger(('morph_analysis',)->morph_extended), finite_form_retagger=FiniteFormRetagger(('morph_extended',)->morph_extended))

In [5]:
text = Text('laulma hüpelnud tantsija').tag_layer()

tagger.tag(text)

text.morph_extended

Layer(name='morph_extended', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech', 'fin'), spans=SL[Span('laulma', [{'normalized_text': 'laulma', 'lemma': 'laulma', 'root': 'laul', 'root_tokens': ['laul'], 'ending': 'ma', 'clitic': '', 'form': 'mod sup ps ill', 'partofspeech': 'V', 'fin': False}, {'normalized_text': 'laulma', 'lemma': 'laulma', 'root': 'laul', 'root_tokens': ['laul'], 'ending': 'ma', 'clitic': '', 'form': 'aux sup ps ill', 'partofspeech': 'V', 'fin': False}, {'normalized_text': 'laulma', 'lemma': 'laulma', 'root': 'laul', 'root_tokens': ['laul'], 'ending': 'ma', 'clitic': '', 'form': 'main sup ps ill', 'partofspeech': 'V', 'fin': False}]),
Span('hüpelnud', [{'normalized_text': 'hüpelnud', 'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ['hüple'], 'ending': 'nud', 'clitic': '', 'form': 'mod indic impf ps neg', 'partofspeech': 'V', 'fin': True}, {'normalized_text': 'hüpelnud', 'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ['hüple'], 'ending': 'nud', 'clitic': '', 'form': 'mod partic past ps', 'partofspeech': 'V', 'fin': False}, {'normalized_text': 'hüpelnud', 'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ['hüple'], 'ending': 'nud', 'clitic': '', 'form': 'aux indic impf ps neg', 'partofspeech': 'V', 'fin': True}, {'normalized_text': 'hüpelnud', 'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ['hüple'], 'ending': 'nud', 'clitic': '', 'form': 'aux partic past ps', 'partofspeech': 'V', 'fin': False}, {'normalized_text': 'hüpelnud', 'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ['hüple'], 'ending': 'nud', 'clitic': '', 'form': 'main indic impf ps neg', 'partofspeech': 'V', 'fin': True}, {'normalized_text': 'hüpelnud', 'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ['hüple'], 'ending': 'nud', 'clitic': '', 'form': 'main partic past ps', 'partofspeech': 'V', 'fin': False}, {'normalized_text': 'hüpelnud', 'lemma': 'hüpelnud', 'root': 'hüpel=nud', 'root_tokens': ['hüpelnud'], 'ending': '0', 'clitic': '', 'form': 'pos', 'partofspeech': 'A', 'fin': None}, {'normalized_text': 'hüpelnud', 'lemma': 'hüpelnud', 'root': 'hüpel=nud', 'root_tokens': ['hüpelnud'], 'ending': '0', 'clitic': '', 'form': 'pos sg nom', 'partofspeech': 'A', 'fin': None}, {'normalized_text': 'hüpelnud', 'lemma': 'hüpelnud', 'root': 'hüpel=nud', 'root_tokens': ['hüpelnud'], 'ending': 'd', 'clitic': '', 'form': 'pos pl nom', 'partofspeech': 'A', 'fin': None}]),
Span('tantsija', [{'normalized_text': 'tantsija', 'lemma': 'tantsija', 'root': 'tantsija', 'root_tokens': ['tantsija'], 'ending': '0', 'clitic': '', 'form': 'com sg nom', 'partofspeech': 'S', 'fin': None}])])

## VerbExtensionSuffixRetagger

In [6]:
# NBVAL_IGNORE_OUTPUT
from estnltk.taggers import VerbExtensionSuffixRetagger

retagger = VerbExtensionSuffixRetagger('morph_analysis')
retagger

VerbExtensionSuffixRetagger(function=<function rewrite at 0x000001EAC693F820>, attr_change=ADD)

In [7]:
text = Text('Laulev hüpelnud tantsija').tag_layer()
retagger.retag(text)
text.morph_analysis

Layer(name='morph_analysis', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech', 'verb_extension_suffix'), spans=SL[Span('Laulev', [{'normalized_text': 'Laulev', 'lemma': 'laulev', 'root': 'laulev', 'root_tokens': ['laulev'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'A', 'verb_extension_suffix': ()}]),
Span('hüpelnud', [{'normalized_text': 'hüpelnud', 'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ['hüple'], 'ending': 'nud', 'clitic': '', 'form': 'nud', 'partofspeech': 'V', 'verb_extension_suffix': ()}, {'normalized_text': 'hüpelnud', 'lemma': 'hüpelnud', 'root': 'hüpel=nud', 'root_tokens': ['hüpelnud'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'A', 'verb_extension_suffix': ('nud',)}, {'normalized_text': 'hüpelnud', 'lemma': 'hüpelnud', 'root': 'hüpel=nud', 'root_tokens': ['hüpelnud'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'A', 'verb_extension_suffix': ('nud',)}, {'normalized_text': 'hüpelnud', 'lemma': 'hüpelnud', 'root': 'hüpel=nud', 'root_tokens': ['hüpelnud'], 'ending': 'd', 'clitic': '', 'form': 'pl n', 'partofspeech': 'A', 'verb_extension_suffix': ('nud',)}]),
Span('tantsija', [{'normalized_text': 'tantsija', 'lemma': 'tantsija', 'root': 'tantsija', 'root_tokens': ['tantsija'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S', 'verb_extension_suffix': ()}])])

## SubcatTagger

In [8]:
from os.path import relpath
from estnltk.taggers import SubcatTagger

fsToSyntFulesFile = relpath(abs_path('taggers/standard/syntax/preprocessing/rules_files/tmorftrtabel.txt'))
subcatFile = relpath(abs_path('taggers/standard/syntax/preprocessing/rules_files/abileksikon06utf.lx'))

tagger = SubcatTagger('morph_analysis', 'morph_extended',
                        fs_to_synt_rules_file=fsToSyntFulesFile, 
                        subcat_rules_file=subcatFile)
tagger

SubcatTagger(input_layers=('morph_analysis',), output_layer=morph_extended, output_attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form',  ..., type: <class 'tuple'>, length: 9, morph_to_syntax_morph_tagger=MorphToSyntaxMorphRetagger(('morph_analysis',)->morph_extended), subcat_retagger=SubcatRetagger(('morph_extended',)->morph_extended))

In [9]:
text = Text('Järel juurduma').tag_layer()
tagger.tag(text)
text.morph_extended

Layer(name='morph_extended', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech', 'subcat'), spans=SL[Span('Järel', [{'normalized_text': 'Järel', 'lemma': 'järel', 'root': 'järel', 'root_tokens': ['järel'], 'ending': '0', 'clitic': '', 'form': 'post', 'partofspeech': 'K', 'subcat': ('gen',)}, {'normalized_text': 'Järel', 'lemma': 'järel', 'root': 'järel', 'root_tokens': ['järel'], 'ending': '0', 'clitic': '', 'form': 'pre', 'partofspeech': 'K', 'subcat': None}]),
Span('juurduma', [{'normalized_text': 'juurduma', 'lemma': 'juurduma', 'root': 'juurdu', 'root_tokens': ['juurdu'], 'ending': 'ma', 'clitic': '', 'form': 'mod sup ps ill', 'partofspeech': 'V', 'subcat': ('Intr',)}, {'normalized_text': 'juurduma', 'lemma': 'juurduma', 'root': 'juurdu', 'root_tokens': ['juurdu'], 'ending': 'ma', 'clitic': '', 'form': 'aux sup ps ill', 'partofspeech': 'V', 'subcat': ('Intr',)}, {'normalized_text': 'juurduma', 'lemma': 'juurduma', 'root': 'juurdu', 'root_tokens': ['juurdu'], 'ending': 'ma', 'clitic': '', 'form': 'main sup ps ill', 'partofspeech': 'V', 'subcat': ('Intr',)}])])

## MorphExtendedTagger

In [10]:
from estnltk.taggers import MorphExtendedTagger

fsToSyntFulesFile = relpath(abs_path('taggers/standard/syntax/preprocessing/rules_files/tmorftrtabel.txt'))
subcat_file = relpath(abs_path('taggers/standard/syntax/preprocessing/rules_files/abileksikon06utf.lx'))

tagger = MorphExtendedTagger(fs_to_synt_rules_file=fsToSyntFulesFile, 
                             allow_to_remove_all=False, 
                             subcat_rules_file=subcat_file)
tagger

MorphExtendedTagger(input_layers=('morph_analysis',), output_layer=morph_extended, output_attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form',  ..., type: <class 'tuple'>, length: 14, punctuation_type_retagger=PunctuationTypeRetagger(('morph_extended',)->morph_extended), morph_to_syntax_morph_retagger=MorphToSyntaxMorphRetagger(('morph_analysis',)->morph_extended), pronoun_type_retagger=PronounTypeRetagger(('morph_extended',)->morph_extended), letter_case_retagger=LetterCaseRetagger(('morph_extended',)->morph_extended), remove_adposition_analyses_retagger=RemoveAdpositionAnalysesRetagger(('morph_extended',)->morph_extended), finite_form_retagger=FiniteFormRetagger(('morph_extended',)->morph_extended), verb_extension_suffix_retagger=VerbExtensionSuffixRetagger(('morph_extended',)->morph_extended), subcat_retagger=SubcatRetagger(('morph_extended',)->morph_extended))

In [11]:
text = Text('Ta on rääkinud!').tag_layer()
tagger.tag(text)
text['morph_extended']

Layer(name='morph_extended', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech', 'punctuation_type', 'pronoun_type', 'letter_case', 'fin', 'verb_extension_suffix', 'subcat'), spans=SL[Span('Ta', [{'normalized_text': 'Ta', 'lemma': 'tema', 'root': 'tema', 'root_tokens': ['tema'], 'ending': '0', 'clitic': '', 'form': 'sg nom', 'partofspeech': 'P', 'punctuation_type': None, 'pronoun_type': ['ps3'], 'letter_case': 'cap', 'fin': None, 'verb_extension_suffix': [], 'subcat': None}]),
Span('on', [{'normalized_text': 'on', 'lemma': 'olema', 'root': 'ole', 'root_tokens': ['ole'], 'ending': '0', 'clitic': '', 'form': 'mod indic pres ps3 sg ps af', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': [], 'subcat': ['Intr']}, {'normalized_text': 'on', 'lemma': 'olema', 'root': 'ole', 'root_tokens': ['ole'], 'ending': '0', 'clitic': '', 'form': 'aux indic pres ps3 sg ps af', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': [], 'subcat': ['Intr']}, {'normalized_text': 'on', 'lemma': 'olema', 'root': 'ole', 'root_tokens': ['ole'], 'ending': '0', 'clitic': '', 'form': 'main indic pres ps3 sg ps af', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': [], 'subcat': ['Intr']}, {'normalized_text': 'on', 'lemma': 'olema', 'root': 'ole', 'root_tokens': ['ole'], 'ending': '0', 'clitic': '', 'form': 'mod indic pres ps3 pl ps af', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': [], 'subcat': ['Intr']}, {'normalized_text': 'on', 'lemma': 'olema', 'root': 'ole', 'root_tokens': ['ole'], 'ending': '0', 'clitic': '', 'form': 'aux indic pres ps3 pl ps af', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': [], 'subcat': ['Intr']}, {'normalized_text': 'on', 'lemma': 'olema', 'root': 'ole', 'root_tokens': ['ole'], 'ending': '0', 'clitic': '', 'form': 'main indic pres ps3 pl ps af', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': [], 'subcat': ['Intr']}]),
Span('rääkinud', [{'normalized_text': 'rääkinud', 'lemma': 'rääkima', 'root': 'rääki', 'root_tokens': ['rääki'], 'ending': 'nud', 'clitic': '', 'form': 'mod indic impf ps neg', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': [], 'subcat': ['Part-P', 'El']}, {'normalized_text': 'rääkinud', 'lemma': 'rääkima', 'root': 'rääki', 'root_tokens': ['rääki'], 'ending': 'nud', 'clitic': '', 'form': 'mod partic past ps', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': False, 'verb_extension_suffix': [], 'subcat': ['Part-P', 'El']}, {'normalized_text': 'rääkinud', 'lemma': 'rääkima', 'root': 'rääki', 'root_tokens': ['rääki'], 'ending': 'nud', 'clitic': '', 'form': 'aux indic impf ps neg', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': [], 'subcat': ['Part-P', 'El']}, {'normalized_text': 'rääkinud', 'lemma': 'rääkima', 'root': 'rääki', 'root_tokens': ['rääki'], 'ending': 'nud', 'clitic': '', 'form': 'aux partic past ps', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': False, 'verb_extension_suffix': [], 'subcat': ['Part-P', 'El']}, {'normalized_text': 'rääkinud', 'lemma': 'rääkima', 'root': 'rääki', 'root_tokens': ['rääki'], 'ending': 'nud', 'clitic': '', 'form': 'main indic impf ps neg', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': [], 'subcat': ['Part-P', 'El']}, {'normal

## CG3 exporter

In [12]:
from estnltk.converters import export_CG3
text = Text('Lähme! Ta on rääkinud.')
text.tag_layer(['sentences','morph_extended'])
export_CG3(text)

['"<s>"',
 '"<Lähme>"',
 '    "mine" Lme V mod indic pres ps1 pl ps af cap <FinV>',
 '    "mine" Lme V aux indic pres ps1 pl ps af cap <FinV>',
 '    "mine" Lme V main indic pres ps1 pl ps af cap <FinV>',
 '"<!>"',
 '    "!" Z Exc',
 '"</s>"',
 '"<s>"',
 '"<Ta>"',
 '    "tema" L0 P pers ps3 sg nom cap',
 '"<on>"',
 '    "ole" L0 V mod indic pres ps3 sg ps af <FinV> <Intr>',
 '    "ole" L0 V aux indic pres ps3 sg ps af <FinV> <Intr>',
 '    "ole" L0 V main indic pres ps3 sg ps af <FinV> <Intr>',
 '    "ole" L0 V mod indic pres ps3 pl ps af <FinV> <Intr>',
 '    "ole" L0 V aux indic pres ps3 pl ps af <FinV> <Intr>',
 '    "ole" L0 V main indic pres ps3 pl ps af <FinV> <Intr>',
 '"<rääkinud>"',
 '    "rääki" Lnud V mod indic impf ps neg <FinV> <Part-P> <El>',
 '    "rääki" Lnud V mod partic past ps <Part-P> <El>',
 '    "rääki" Lnud V aux indic impf ps neg <FinV> <Part-P> <El>',
 '    "rääki" Lnud V aux partic past ps <Part-P> <El>',
 '    "rääki" Lnud V main indic impf ps neg <FinV> <Par